In [97]:
# imports
%load_ext autoreload
%autoreload 2

import numpy as np
from collections import OrderedDict

# import hack to allow this example file to load the SMD package without installing it
import sys
sys.path.append("SMD/")

import smd as SMD

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Setup model

In [98]:
from types import SimpleNamespace
class Model():
    def __init__(self):
        
        self.par = SimpleNamespace()
        par = self.par
        par.a = 2
        par.b = 1
        
        par.Nsim = 200
        
        self.sol = SimpleNamespace()
        self.sim = SimpleNamespace()
        
    def update_par(self,theta,names):
        for name,val in zip(names,theta):
            setattr(self.par,name,val)
    
    def solve(self):
        self.sol.y = lambda x: self.par.a + self.par.b * x
        
    def setup_simulation(self,seed=2021):
        np.random.seed(seed)
        self.sim.draws = np.random.normal(size=self.par.Nsim)
        
    def simulate(self):
        
        self.sim.x = self.sim.draws
        self.sim.y = self.sol.y(self.sim.x)
        
    def calc_moments(self):
        moms = OrderedDict()
        moms['avg'] = np.mean(self.sim.y)
        moms['cov'] = np.cov(self.sim.y,self.sim.x)[0,1]
        
        return moms
        

## Simulate synthetic data

In [99]:
data = Model()
data.solve()
data.setup_simulation(seed=2021)
data.simulate()

moms = data.calc_moments()
moms

OrderedDict([('avg', 2.083390661379895), ('cov', 0.9907806084430754)])

## setup model and estimate $a$ and $b$

In [100]:
model = Model()
model.par.Nsim = 500
model.setup_simulation(seed=2020)

In [101]:
smd = SMD.SMDClass()
# skip loading data and just insert datamoms for now
smd.datamoms = moms

In [109]:
# Estimation setup
est_par = {
    'a': {'init':2.0,'bounds':[1.0 , 3.0]},
    'b': {'init':1.0,'bounds':[0.1 , 2.0]},
}

res = smd.estimate(model,est_par)

         Current function value: -1.181578
         Iterations: 19
         Function evaluations: 527
         Gradient evaluations: 173


In [110]:
res

      fun: -1.181578411361275
 hess_inv: array([[ 4.89491830e-05, -6.22556291e-06],
       [-6.22556291e-06,  1.26921912e-05]])
      jac: array([-0.00029868,  0.00015272])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 527
      nit: 19
     njev: 173
   status: 2
  success: False
        x: array([0.99999998, 2.        ])